In [1]:
import numpy as np 
import pandas as pd 
import pickle

# visualizatioion pkg 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_format='retina'

import sys
sys.path.append('..')

from utils.viz import viz 
from utils.model import *  
from utils.fig_fn import *
viz.get_style()

In [5]:
def get_table1(exp, agents, method='mle'):

    agents = agents if exp=='exp1' else [m+'_fea' for m in agents] 
    crs = {}
    for m in agents:
        n_param = eval(m).n_params
        fname = f'{pth}/../analyses/{exp}/{m}/{method}-eval.csv'
        data  = pd.read_csv(fname)
        data['nll'] = -data['ll']

        group_data = data
        subj_Lst = group_data['sub_id'].unique()
        nlls, aics, bics = [], [], [] 
        ass_nll, ass_aic, ass_bic = [], [], []
        gen_nll, gen_aic, gen_bic = [], [], []
        for sub_id in subj_Lst:
            inll = group_data.query(f'sub_id=={sub_id}')['nll'].sum() 
            inum = group_data.query(f'sub_id=={sub_id}')['nll'].shape[0]
            nlls.append(inll)
            aics.append(2*inll + 2*n_param)
            bics.append(2*inll + np.log(inum)*n_param)

            anll = group_data.query(f'sub_id=={sub_id} & stage=="train"'
                                )['nll'].sum() 
            anum = group_data.query(f'sub_id=={sub_id} & stage=="train"'
                                )['nll'].shape[0]
            ass_nll.append(anll)
            ass_aic.append(2*anll + 2*n_param)
            ass_bic.append(2*anll + np.log(anum)*n_param)

            gnll = group_data.query(f'sub_id=={sub_id} & stage=="test"'
                                )['nll'].sum() 
            gnum = group_data.query(f'sub_id=={sub_id} & stage=="train"'
                                )['nll'].shape[0]
            gen_nll.append(gnll)
            gen_aic.append(2*gnll + 2*n_param)
            gen_bic.append(2*gnll + np.log(gnum)*n_param)
        
        mat = {'tot_nll': nlls, 'tot_aic': aics, 'tot_bic': bics, 
               'ass_nll': ass_nll, 'ass_aic': ass_aic, 'ass_bic': ass_bic,
               'gen_nll': gen_nll, 'gen_aic': gen_aic, 'gen_bic': gen_bic}
        
        for k, v in mat.items(): mat[k] = np.mean(v)
        crs[m] = mat
        
        print(f'''
             # ------------------------------- {m} --------------------------------- #
                 Overall nll: {np.mean(nlls):.3f}      aic: {np.mean(aics):.3f}      bic: {np.mean(bics):.3f}
                 Train nll: {np.mean(ass_nll):.3f}         aic: {np.mean(ass_aic):.3f}      bic: {np.mean(ass_bic):.3f}            
                 Test nll: {np.mean(gen_nll):.3f}          aic: {np.mean(gen_aic):.3f}       bic: {np.mean(gen_bic):.3f} 
         ''')
    
    return pd.DataFrame.from_dict(crs).round(3)

In [6]:
table1 = get_table1('exp1', ['rmPG', 'caPG', 'ecPG'], 'map')


             # ------------------------------- rmPG --------------------------------- #
                 Overall nll: 114.562      aic: 231.124      bic: 234.499
                 Train nll: 66.196         aic: 134.392      bic: 137.179            
                 Test nll: 48.366          aic: 98.732       bic: 101.520 
         

             # ------------------------------- caPG --------------------------------- #
                 Overall nll: 111.160      aic: 226.319      bic: 233.070
                 Train nll: 64.235         aic: 132.470      bic: 138.045            
                 Test nll: 46.925          aic: 97.850       bic: 103.425 
         

             # ------------------------------- ecPG --------------------------------- #
                 Overall nll: 101.202      aic: 208.403      bic: 218.529
                 Train nll: 62.061         aic: 130.121      bic: 138.484            
                 Test nll: 39.141          aic: 84.282       bic: 92.644 
         

In [7]:
table1.to_csv('../figures/exp1/Table1.csv')

,rmPG,caPG,ecPG
tot_nll,114.562,111.160,101.202
tot_aic,231.124,226.319,208.403
tot_bic,234.499,233.070,218.529
ass_nll,66.196,64.235,62.061
ass_aic,134.392,132.470,130.121
ass_bic,137.179,138.045,138.484
gen_nll,48.366,46.925,39.141
gen_aic,98.732,97.850,84.282
gen_bic,101.520,103.425,92.644
